In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from random import choice
import os
import scipy as sp
import scipy.fftpack
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter


In [2]:
def get_branch(row):
    if row['Plate']==94:
        hyph = row['end']
        name = f'branches/ts_{hyph}_auto.npy'
        if not os.path.isfile(name):
            return(None)
        else:
            t_real = np.load(name)
            return(row['time_since_emergence'] in t_real)

In [3]:
time_plate_info = pd.read_pickle("time_plate_infos.pick")
global_hypha_info = pd.read_pickle("global_hypha_info.pick")
time_hypha_info = pd.read_pickle("time_hypha_infos.pick")
global_hypha_info['lapse']=global_hypha_info['time_stop_growth']-global_hypha_info['time_init_growth']
time_hypha_info['distance_final_pos_smooth'] = time_hypha_info['distance_final_pos'].rolling(window = 5).mean()
time_hypha_info['new_branch'] = time_hypha_info.apply(get_branch, axis=1)

In [4]:
plate = 94
select_time_plate = time_plate_info.loc[time_plate_info['Plate']==plate]
select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
select_global = global_hypha_info.loc[global_hypha_info['Plate']==plate]
select_global = select_global.loc[select_global['lapse']>10]
hyphae_interest= select_global['hypha'].unique()
# hyphae_select= [hyph for hyph in hyphae_interest if len(select_time.loc[(select_time['end']==hyph)&(select_time['has_reached_final_pos']=='False')])]
select_time = select_time.loc[select_time['end'].isin(hyphae_interest)]
# select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
select_time = select_time.loc[select_time['speed'].between(0,400)]
select_time = select_time.loc[select_time['distance_final_pos']>100]

select_time = select_time.loc[select_time['in_ROI']=='True']

In [5]:
actual_list_hyph = list(np.load('actual_list_hyph.npy'))
actual_list_hyph.remove(57138)


In [6]:
total_deltas = []
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
for hyph in actual_list_hyph:
    t_real = np.load(f'branches/ts_{hyph}_auto.npy')
    t_real.sort()
    deltas = np.diff(t_real)    
    total_deltas += list(deltas)

In [7]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(total_deltas,np.array(range(0,26))-0.5)
ax.set_xlabel('time delta between two branching events (h)')
ax.set_ylabel('count')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'count')

In [191]:
actual_list_hyph = list(np.load('actual_list_hyph.npy'))
actual_list_hyph.remove(57138)
total_distances = []
estimated_timedelta = []
for hyph in actual_list_hyph:
    distances = np.load(f'branches/distances_{hyph}_auto.npy')
    t_real = np.load(f'branches/ts_{hyph}_auto.npy')
    t_real.sort()
    table = select_time.loc[select_time['end']==hyph]
    for index,t in enumerate(t_real[1:-1]):
        av_speed=np.mean(table.loc[table['time_since_emergence'].between(t,t_real[index+1],inclusive = 'left')]['speed'])
        if not np.isnan(av_speed):
            estimated = distances[index]/av_speed
            estimated_timedelta.append(distances[index]/av_speed)
    total_distances += list(distances)

In [9]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(total_distances,np.linspace(0,3000,15))
ax.set_xlabel('distance between two branching events ($\mu m$)')
ax.set_ylabel('count')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'count')

In [64]:
np.mean(total_distances)

1034.571053985846

In [192]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(estimated_timedelta,np.array(range(0,26))-0.5)
ax.set_xlabel('time delta between two branching events (estimated) (h)')
ax.set_ylabel('count')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'count')

In [12]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
actual_list_hyph = list(np.load('actual_list_hyph.npy'))
actual_list_hyph.remove(57138)

for hyph in actual_list_hyph:
    total_distances = []
    estimated_timedelta = []
    distances = np.load(f'branches/distances_{hyph}_auto.npy')
    t_real = np.load(f'branches/ts_{hyph}_auto.npy')
    t_real.sort()
    print(len(distances),len(t_real))
    table = select_time.loc[select_time['end']==hyph]
    for index,t in enumerate(t_real[1:-1]):
        av_speed=np.mean(table.loc[table['time_since_emergence'].between(t,t_real[index+1],inclusive = 'left')]['speed'])
        if not np.isnan(av_speed):
            estimated = distances[index]/av_speed
            estimated_timedelta.append(distances[index]/av_speed)
    total_distances += list(distances)
    ax.plot(np.cumsum(estimated_timedelta),estimated_timedelta)
    break

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

5 7


In [57]:
for hyph in actual_list_hyph:
    t_real = np.load(f'branches/ts_{hyph}_auto.npy')
    if len(t_real)>10:
        print(hyph)

0
1
139
9612
19
164
24489
41036
6992
18897
10617


In [7]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
actual_list_hyph = list(np.load('actual_list_hyph.npy'))
actual_list_hyph.remove(57138)
hyph = 6992
total_distances = []
estimated_timedelta = []
distances = np.load(f'branches/distances_{hyph}_auto.npy')
t_real = np.load(f'branches/ts_{hyph}_auto.npy')
t_real.sort()
deltas = np.diff(t_real) 
print(len(distances),len(t_real))
table = select_time.loc[select_time['end']==hyph]
for index,t in enumerate(t_real[1:-1]):
    av_speed=np.mean(table.loc[table['time_since_emergence'].between(t,t_real[index+1],inclusive = 'left')]['speed'])
    if not np.isnan(av_speed):
        estimated = distances[index]/av_speed
        estimated_timedelta.append(distances[index]/av_speed)
total_distances += list(distances)
# ax.plot(t_real[2:-1],estimated_timedelta)
ax.plot(np.cumsum(estimated_timedelta),estimated_timedelta)
ax.set_xlabel('time since emergence of the hypha (h)')
ax.set_ylabel('time until next branching event (h)')
# ax.plot(t_real[:-1],deltas)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

19 21


Text(0, 0.5, 'time until next branching event (h)')

In [178]:
f = interp1d(np.cumsum(estimated_timedelta), estimated_timedelta)
deb = np.min(np.cumsum(estimated_timedelta))
end = np.max(np.cumsum(estimated_timedelta))
frequ = 1
x = np.linspace(deb,end,int((end-deb)*frequ)+1)
resampled = pd.DataFrame({'time':x,'estimated_timedelta':f(x)})
resampled['estimated_timedelta_ZMUV'] = (resampled['estimated_timedelta']-np.mean(resampled['estimated_timedelta']))/np.std(resampled['estimated_timedelta'])

In [179]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
# ax.plot(x,resampled['speed'])
# ax.plot(x,reg.predict(x.reshape((-1, 1))))
ax.plot(x,resampled['estimated_timedelta_ZMUV'])
ax.set_xlabel('time since emergence of the hypha (h)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'time since emergence of the hypha (h)')

In [180]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

In [182]:
51/6

8.5

In [181]:
plt.close("all")
s = resampled[~np.isnan(resampled['estimated_timedelta_ZMUV'].astype(np.float))]['estimated_timedelta_ZMUV'].values
autoc = autocorr(s)
autoc = autoc/autoc.max()
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(autoc)
ax.set_xlabel('lag (h)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'lag (h)')

In [183]:
speed_fft = sp.fftpack.fft(s)
speed_psd = np.abs(speed_fft) ** 2
fftfreq = sp.fftpack.fftfreq(len(speed_psd))
i = fftfreq > 0
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(24*frequ*fftfreq[i], speed_psd[i])
ax.set_xlabel('Frequency (day-1)')
ax.set_ylabel('PSD')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'PSD')

In [184]:
24/2.74

8.75912408759124

In [6]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
size = 48
fourrier = pd.DataFrame(columns = ['frequ','amp','hyph'])
for hyph in actual_list_hyph:
    t_real = np.load(f'branches/ts_{hyph}_auto.npy')
    total_distances = []
    estimated_timedelta = []
    distances = np.load(f'branches/distances_{hyph}_auto.npy')
    t_real = np.load(f'branches/ts_{hyph}_auto.npy')
    t_real.sort()
    deltas = np.diff(t_real) 
    table = select_time.loc[select_time['end']==hyph]
    for index,t in enumerate(t_real[1:-1]):
        av_speed=np.mean(table.loc[table['time_since_emergence'].between(t,t_real[index+1],inclusive = 'left')]['speed'])
        if not np.isnan(av_speed):
            estimated = distances[index]/av_speed
            estimated_timedelta.append(distances[index]/av_speed)
    total_distances += list(distances)
    if np.sum(estimated_timedelta)>size and hyph !=139:
        f = interp1d(np.cumsum(estimated_timedelta), estimated_timedelta)
        deb = np.min(np.cumsum(estimated_timedelta))
        end = np.max(np.cumsum(estimated_timedelta))
        frequ = 1
        x = np.linspace(deb,end,int((end-deb)*frequ)+1)
        if len(x)>size:
            resampled = pd.DataFrame({'time':x,'estimated_timedelta':f(x)})
            resampled['estimated_timedelta_ZMUV'] = (resampled['estimated_timedelta']-np.mean(resampled['estimated_timedelta']))/np.std(resampled['estimated_timedelta'])
            s = resampled[~np.isnan(resampled['estimated_timedelta_ZMUV'].astype(np.float))]['estimated_timedelta_ZMUV'].values
            s = s[:size]
            print(len(s))
            speed_fft = sp.fftpack.fft(s)
            speed_psd = np.abs(speed_fft) ** 2
            fftfreq = sp.fftpack.fftfreq(len(speed_psd))
            i = fftfreq > 0
            data = [{'frequ' : fftfreq[j]*frequ,'amp' : speed_psd[j],'hyph' : hyph} for j,v in enumerate(i) if v]
            fourrier=fourrier.append(data,ignore_index=True)
            ax.plot(24*frequ*fftfreq[i], speed_psd[i],label=f'hypha {hyph}')
            ax.set_xlabel('Frequency (day-1)')
            ax.set_ylabel('PSD')
            
plt.legend()
#             break
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
df = pd.concat([fourrier.groupby('frequ')['amp'].mean()], axis=1).copy()
df = df.sort_index()
ax.plot(24*df.index,df['amp'],label=plate)
ax.set_xlabel('Frequency (day-1)')
ax.set_ylabel('PSD')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

48
48
48
48
48


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'PSD')

In [166]:
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
for size in range(62,70,2):
    fourrier = pd.DataFrame(columns = ['frequ','amp','hyph'])
    for hyph in actual_list_hyph:
        t_real = np.load(f'branches/ts_{hyph}_auto.npy')
        total_distances = []
        estimated_timedelta = []
        distances = np.load(f'branches/distances_{hyph}_auto.npy')
        t_real = np.load(f'branches/ts_{hyph}_auto.npy')
        t_real.sort()
        deltas = np.diff(t_real) 
        table = select_time.loc[select_time['end']==hyph]
        for index,t in enumerate(t_real[1:-1]):
            av_speed=np.mean(table.loc[table['time_since_emergence'].between(t,t_real[index+1],inclusive = 'left')]['speed'])
            if not np.isnan(av_speed):
                estimated = distances[index]/av_speed
                estimated_timedelta.append(distances[index]/av_speed)
        total_distances += list(distances)
        if np.sum(estimated_timedelta)>size and hyph !=139:
            f = interp1d(np.cumsum(estimated_timedelta), estimated_timedelta)
            deb = np.min(np.cumsum(estimated_timedelta))
            end = np.max(np.cumsum(estimated_timedelta))
            frequ = 1
            x = np.linspace(deb,end,int((end-deb)*frequ)+1)
            if len(x)>size:
                print(hyph)
                resampled = pd.DataFrame({'time':x,'estimated_timedelta':f(x)})
                resampled['estimated_timedelta_ZMUV'] = (resampled['estimated_timedelta']-np.mean(resampled['estimated_timedelta']))/np.std(resampled['estimated_timedelta'])
                s = resampled[~np.isnan(resampled['estimated_timedelta_ZMUV'].astype(np.float))]['estimated_timedelta_ZMUV'].values
                s = s[:size]
#                 print(len(s))
                speed_fft = sp.fftpack.fft(s)
                speed_psd = np.abs(speed_fft) ** 2
                fftfreq = sp.fftpack.fftfreq(len(speed_psd))
                i = fftfreq > 0
                data = [{'frequ' : fftfreq[j]*frequ,'amp' : speed_psd[j],'hyph' : hyph} for j,v in enumerate(i) if v]
                fourrier=fourrier.append(data,ignore_index=True)
    #             ax.plot(24*frequ*fftfreq[i], speed_psd[i],label=hyph)
    #             ax.set_xlabel('Frequency (day-1)')
    #             ax.set_ylabel('PSD')

    df = pd.concat([fourrier.groupby('frequ')['amp'].mean()], axis=1).copy()
    df = df.sort_index()
    ax.plot(24*df.index,df['amp'],label=size)
    ax.set_xlabel('Frequency (day-1)')
    ax.set_ylabel('PSD')
    plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1
19
164
1
19
164
1
19
164
1
19
164


In [188]:
24/3.50

6.857142857142857